In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/bias-bench
!python -m pip install -e .

Mounted at /content/drive
/content/drive/MyDrive/bias-bench
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/drive/MyDrive/bias-bench
     |██████████████████████████████▎ | 834.1 MB 1.2 MB/s eta 0:00:42tcmalloc: large alloc 1147494400 bytes == 0x3a364000 @  0x7f44c8f8b615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |████████████████████████████████| 881.9 MB 1.9 kB/s 
     |████████████████████████████████| 3.5 MB 55.6 MB/s 
     |████████████████████████████████| 38.1 MB 1.2 MB/s 
     |████████████████████████████████| 1.5 MB 45.3 MB/s 
     |████████████████████████████████| 311 kB 73.4 MB/s 
     |████████████████████████████████| 58 kB 7.5 MB/s 
     |███████████████████████████

In [ ]:
from transformers import GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [ ]:
len(tokenizer("The following text discriminates against people because of their gender: ")["input_ids"])

13

In [2]:
settings = ['white', 'tokenized', 'nouns', 'asian_nouns', 'white_nouns', 'black_nouns', 'hispanic_nouns']

In [ ]:
#evaluate original bias
setting = 'nounx'
for setting in settings:
  for i in range(0,5):
    !python experiments/stereoset.py --input_file "./data/stereoset/gender_data_{setting}_{i}.json" --model "GPT2LMHeadModel" --model_name_or_path "gpt2" --seed 1 --experiment_name 'implicit_gender_{setting}_{i}'

Running StereoSet:
 - input_file: ./data/stereoset/gender_data_nounx_0.json
 - persistent_dir: /content/drive/MyDrive/bias-bench
 - model: GPT2LMHeadModel
 - model_name_or_path: gpt2
 - batch_size: 1
 - seed: 1
 - experiment_id : stereoset_m-GPT2LMHeadModel_c-gpt2_s-1
 - experiment_name : implicit_gender_nounx_0
Using mask_token, but it is not set yet.
Evaluating intrasentence task.
100% 523/523 [00:09<00:00, 53.01it/s]
Running StereoSet:
 - input_file: ./data/stereoset/gender_data_nounx_1.json
 - persistent_dir: /content/drive/MyDrive/bias-bench
 - model: GPT2LMHeadModel
 - model_name_or_path: gpt2
 - batch_size: 1
 - seed: 1
 - experiment_id : stereoset_m-GPT2LMHeadModel_c-gpt2_s-1
 - experiment_name : implicit_gender_nounx_1
Using mask_token, but it is not set yet.
Evaluating intrasentence task.
100% 523/523 [00:09<00:00, 53.05it/s]
Running StereoSet:
 - input_file: ./data/stereoset/gender_data_nounx_2.json
 - persistent_dir: /content/drive/MyDrive/bias-bench
 - model: GPT2LMHeadMod

In [ ]:
#print original bias scores
for i in range(0,5):
    !python experiments/stereoset_evaluation.py --predictions_file results/stereoset/stereoset_m-GPT2LMHeadModel_c-gpt2_s-1_implicit_gender_{setting}_{i}.json   --output_file results/stereoset/results_implicit_gender_{setting}_{i}.json --data_file "./data/stereoset/gender_data_{setting}_{i}.json"

Evaluating StereoSet files:
 - predictions_file: results/stereoset/stereoset_m-GPT2LMHeadModel_c-gpt2_s-1_implicit_gender_nounx_0.json
 - predictions_dir: None
 - output_file: results/stereoset/results_implicit_gender_nounx_0.json
 - data_file: ./data/stereoset/gender_data_nounx_0.json
intrasentence
	implicit_gender
		Count: 1046.0
		LM Score: 0.0
		SS Score: 55.831739961759084
		ICAT Score: 0.0
Evaluating StereoSet files:
 - predictions_file: results/stereoset/stereoset_m-GPT2LMHeadModel_c-gpt2_s-1_implicit_gender_nounx_1.json
 - predictions_dir: None
 - output_file: results/stereoset/results_implicit_gender_nounx_1.json
 - data_file: ./data/stereoset/gender_data_nounx_1.json
intrasentence
	implicit_gender
		Count: 1046.0
		LM Score: 0.0
		SS Score: 60.9942638623327
		ICAT Score: 0.0
Evaluating StereoSet files:
 - predictions_file: results/stereoset/stereoset_m-GPT2LMHeadModel_c-gpt2_s-1_implicit_gender_nounx_2.json
 - predictions_dir: None
 - output_file: results/stereoset/results_im

In [ ]:
#debias
prompt = {"gender": "The following text discriminates against people because of their gender identity: "}
prompt_str = str(prompt).replace("\'", "\"")
print(prompt_str)
print(prompt["gender"])
prompt_len = len(tokenizer(prompt["gender"])["input_ids"])
print(prompt_len)
for setting in settings:
    for i in range(0,5):
        !python experiments/stereoset_debias.py --input_file "./data/stereoset/gender_data_{setting}_{i}.json" --model "SelfDebiasGPT2LMHeadModel" --model_name_or_path "gpt2" --seed 1 --experiment_name 'implicit_gender_{setting}_{i}' --debiasing_prefixes '{prompt_str}' --bias_position {prompt_len}
       

{"gender": "The following text discriminates against people because of their gender identity: "}
The following text discriminates against people because of their gender identity: 
14
Running StereoSet:
 - persistent_dir: /content/drive/MyDrive/bias-bench
 - model: SelfDebiasGPT2LMHeadModel
 - model_name_or_path: gpt2
 - bias_direction: None
 - projection_matrix: None
 - load_path: None
 - batch_size: 1
 - bias_type: gender
 - seed: 1
 - experiment_id : stereoset_m-SelfDebiasGPT2LMHeadModel_c-gpt2_t-gender_s-1
 - experiment_name : implicit_gender_white_0
 - debiasing prefixes : {"gender": "The following text discriminates against people because of their gender identity: "}
Using mask_token, but it is not set yet.
Evaluating intrasentence task.
100% 523/523 [00:20<00:00, 25.09it/s]
Running StereoSet:
 - persistent_dir: /content/drive/MyDrive/bias-bench
 - model: SelfDebiasGPT2LMHeadModel
 - model_name_or_path: gpt2
 - bias_direction: None
 - projection_matrix: None
 - load_path: None
 - 

In [ ]:
#evaluate debias
for setting in settings:
    for i in range(0,5):
        !python experiments/stereoset_evaluation.py --predictions_file results/stereoset/stereoset_m-SelfDebiasGPT2LMHeadModel_c-gpt2_t-gender_s-1_implicit_gender_{setting}_{i}.json   --output_file results/stereoset/results_implicit_gender_{setting}_{i}.json --data_file "./data/stereoset/gender_data_{setting}_{i}.json"

Evaluating StereoSet files:
 - predictions_file: results/stereoset/stereoset_m-SelfDebiasGPT2LMHeadModel_c-gpt2_t-gender_s-1_implicit_gender_nounx_0.json
 - predictions_dir: None
 - output_file: results/stereoset/results_implicit_gender_nounx_0.json
 - data_file: ./data/stereoset/gender_data_nounx_0.json
intrasentence
	implicit_gender
		Count: 1046.0
		LM Score: 0.0
		SS Score: 50.2868068833652
		ICAT Score: 0.0
Evaluating StereoSet files:
 - predictions_file: results/stereoset/stereoset_m-SelfDebiasGPT2LMHeadModel_c-gpt2_t-gender_s-1_implicit_gender_nounx_1.json
 - predictions_dir: None
 - output_file: results/stereoset/results_implicit_gender_nounx_1.json
 - data_file: ./data/stereoset/gender_data_nounx_1.json
intrasentence
	implicit_gender
		Count: 1046.0
		LM Score: 0.0
		SS Score: 51.05162523900574
		ICAT Score: 0.0
Evaluating StereoSet files:
 - predictions_file: results/stereoset/stereoset_m-SelfDebiasGPT2LMHeadModel_c-gpt2_t-gender_s-1_implicit_gender_nounx_2.json
 - prediction

Print scores per setting

In [3]:
import json
import numpy as np

In [4]:
result_dict = {}
for setting in settings:
    result_dict[setting] = []
    for i in range(0,5):

      experiment_name = f'implicit_gender_{setting}_{i}'
      file_name = f'results/stereoset/results_{experiment_name}.json'
      test_name = f'stereoset_m-SelfDebiasGPT2LMHeadModel_c-gpt2_t-gender_s-1_{experiment_name}'
      with open(file_name, "r") as read_file:
          
          res = json.load(read_file)
      score = res[test_name]['intrasentence']['implicit_gender']['SS Score']
      result_dict[setting].append(score)

In [5]:
for setting in result_dict.keys():
  
  print(f"{setting} {np.mean(result_dict[setting])}" )

white 47.76290630975144
tokenized 48.33652007648183
nouns 46.15678776290631
asian_nouns 37.05544933078394
white_nouns 46.19502868068834
black_nouns 51.586998087954115
hispanic_nouns 62.48565965583174
